In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

We load the results from snippy:

In [ ]:
df_mut = pd.read_csv("../Data/mutations_all.csv.gz", dtype={"strain": str, "ref_strain": str})
df_mut

Next, we count the number of unique mutations per condition and per strain - reference strain pairing. We also pivot the table for easier plotting using a heatmap:

In [ ]:
df_mut_num = (
    df_mut
    .groupby(["condition", "strain", "ref_strain"], as_index=False)
    .nunique()
    .pivot(index=["condition", "ref_strain"], columns="strain", values="GENE")
    .reset_index()
    .sort_values(["condition", "ref_strain"], ascending=[False, False])
)

Finally, we plot:

In [ ]:
def draw_heatmap(*args, **kwargs):
    data = kwargs.pop('data')
    d = data.drop(columns="condition").set_index("ref_strain")
    return sns.heatmap(d, **kwargs)

g = (
    sns.FacetGrid(df_mut_num, col="condition", height=4)
    .map_dataframe(draw_heatmap, cmap="viridis", annot=True, fmt=".0f")
    .set_titles(col_template="{col_name}")
    .set(xlabel="Strain", ylabel="Ref. strain")
)